In [1]:
import chainer as ch
from chainer import datasets
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions

import numpy as np
mushroomsfile = 'mushrooms.csv'
data_array = np.genfromtxt(mushroomsfile, delimiter=',', dtype=str, skip_header=1)

for col in range(data_array.shape[1]):
    data_array[:, col] = np.unique(data_array[:, col], return_inverse=True)[1]

In [10]:
data_array[0:10]

array([['1', '5', '2', '4', '1', '6', '1', '0', '1', '4', '0', '3', '2',
        '2', '7', '7', '0', '2', '1', '4', '2', '3', '5'],
       ['0', '5', '2', '9', '1', '0', '1', '0', '0', '4', '0', '2', '2',
        '2', '7', '7', '0', '2', '1', '4', '3', '2', '1'],
       ['0', '0', '2', '8', '1', '3', '1', '0', '0', '5', '0', '2', '2',
        '2', '7', '7', '0', '2', '1', '4', '3', '2', '3'],
       ['1', '5', '3', '8', '1', '6', '1', '0', '1', '5', '0', '3', '2',
        '2', '7', '7', '0', '2', '1', '4', '2', '3', '5'],
       ['0', '5', '2', '3', '0', '5', '1', '1', '0', '4', '1', '3', '2',
        '2', '7', '7', '0', '2', '1', '0', '3', '0', '1'],
       ['0', '5', '3', '9', '1', '0', '1', '0', '0', '5', '0', '2', '2',
        '2', '7', '7', '0', '2', '1', '4', '2', '2', '1'],
       ['0', '0', '2', '8', '1', '0', '1', '0', '0', '2', '0', '2', '2',
        '2', '7', '7', '0', '2', '1', '4', '2', '2', '3'],
       ['0', '0', '3', '8', '1', '3', '1', '0', '0', '5', '0', '2', '2',
   

In [2]:
X = data_array[:, 1:].astype(np.float32) #  extract all columns but the first from all rows
Y = data_array[:, 0].astype(np.int32)[:, None] # extract only the first column and put each element into an array of its own
train, test = datasets.split_dataset_random(
    datasets.TupleDataset(X, Y), int(data_array.shape[0] * .7))

In [25]:
Y[0:10]

array([[1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0]], dtype=int32)

In [6]:
train_iter = ch.iterators.SerialIterator(train, 100)
test_iter = ch.iterators.SerialIterator(
    test, 100, repeat=False, shuffle=False)


# Network definition
def MLP(n_units, n_out):
    layer = ch.Sequential(L.Linear(n_units), F.relu)
    model = layer.repeat(2)
    model.append(L.Linear(n_out))

    return model

    def __call__(self,x):
        print(x)
        return model(x)


model = L.Classifier(
    MLP(44, 1), lossfun=F.sigmoid_cross_entropy, accfun=F.binary_accuracy)

# Setup an optimizer
optimizer = ch.optimizers.SGD().setup(model)

# Create the updater, using the optimizer
updater = training.StandardUpdater(train_iter, optimizer, device=-1)

# Set up a trainer
trainer = training.Trainer(updater, (1, 'epoch'), out='result')
# Evaluate the model with the test dataset for each epoch
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))

# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

trainer.extend(extensions.snapshot(), trigger=(2, 'epoch'))

# Write a log of evaluation statistics for each epoch
trainer.extend(extensions.LogReport())

# Save two plot images to the result dir
if extensions.PlotReport.available():
    trainer.extend(
        extensions.PlotReport(['main/loss', 'validation/main/loss'],
                              'epoch', file_name='loss.png'))
    trainer.extend(
        extensions.PlotReport(
            ['main/accuracy', 'validation/main/accuracy'],
            'epoch', file_name='accuracy.png'))

# Print selected entries of the log to stdout
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

#  Run the training
trainer.run()

# x, t = test[np.random.randint(len(test))]

# predict = model.predictor(x[None]).data
# predict = predict[0][0]

# if predict >= 0:
#     print('Predicted Poisonous, Actual ' + ['Edible', 'Poisonous'][t[0]])
# else:
#     print('Predicted Edible, Actual ' + ['Edible', 'Poisonous'][t[0]])


/home/felix.wolff2/.local/lib/python3.6/site-packages/chainer/training/extensions/plot_report.py:25: UserWarning: matplotlib is not installed on your environment, so nothing will be plotted at this time. Please install matplotlib to plot figures.

  $ pip install matplotlib

  warnings.warn('matplotlib is not installed on your environment, '


epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           0.557323    0.48885               0.729123       0.781684                  0.251866      
2           0.434331    0.435051              0.828772       0.818632                  0.475131      
3           0.390184    0.406244              0.845439       0.841832                  0.740202      
4           0.356114    0.371526              0.862982       0.851432                  0.976747      
5           0.331505    0.351207              0.873684       0.868232                  1.22832       
